# Importaciones relevantes

In [1]:
import requests #para llamar a la API
import pandas #librería para trabajar con tablas
from io import StringIO # para leer texto y pasarlo a pandas
pandas.options.plotting.backend = "plotly"


# Usando el Meteorology Query de La Silla

Los datos vienen en formato`csv` o valores separados por comas (**c**oma **s**eparated **v**alue) y lo alimentaremos a nuestra herramienta `pandas` para leerlo como tablar. En este bloque de código se obtienen datospara un rango de fechas `start_date` yy`end_date`, además de especificar el máximo de filas que se devolverán en la llamada `max_rows_returned`. Los datos quedan almacenados en la tabla `weather_data`..

In [2]:
start_date = "2025-09-01"
end_date = "2025-09-20"
max_rows_returned = 20000
url = f"https://archive.eso.org/wdb/wdb/asm/meteo_lasilla/query?wdbo=csv&max_rows_returned={max_rows_returned}&start_date={start_date}..{end_date}"
# url = f'https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m,precipitation_probability'

_response = requests.get(url)
response = _response.text
weather_data = pandas.read_table(StringIO(response),sep=",",index_col=0).dropna(axis=1,how="all") # dropna borra los valores NaN de la tabla
weather_data.describe() # muestra las primeras cinco filas

,Air Pressure at 2m [hPa],Ambient Temperature at 2m [C],Dew Temperature at 2m [C],Relative Humidity at 2m [%],Wind Direction at 10m [deg],Wind Speed at 10m [m/s]
count,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,769.32744,10.734745,-5.635275,18.149900,68.884450,8.560755
std,1.64387,4.190231,2.910737,12.189121,90.373276,3.818203
min,766.50000,1.300000,-11.300000,1.000000,0.000000,0.100000
25%,768.00000,7.400000,-8.200000,10.000000,24.000000,5.900000
50%,768.90000,9.800000,-5.900000,15.000000,41.000000,8.600000
75%,770.80000,14.700000,-3.100000,24.000000,54.000000,11.100000
max,773.00000,19.500000,1.500000,83.000000,359.000000,24.200000


La lista completa de parámetros que puede recibir la query es: `max_rows_returned=10^&start_date=^%^3E2025-07-15^&integration=^&tab_press=on^&press=^&tab_presqnh=on^&presqnh=^&tab_temp1=on^&temp1=^&tab_temp2=on^&temp2=^&tab_temp3=on^&temp3=^&tab_tempdew2=on^&tempdew2=^&tab_rhum2=on^&rhum2=^&tab_wind_dir1=on^&wind_dir1=^&tab_wind_dir2=on^&wind_dir2=^&tab_wind_speed1=on^&wind_speed1=^&tab_wind_speed2=on^&wind_speed2=^&order=start_date^`

In [ ]:
weather_data.plot()

Dado que los nombres de las variables son muy largas, vamos a introducir algunos *alias* para llamarlas con facilidad

In [ ]:
air_pressure = "Air Pressure at 2m [hPa]"
temperature = "Ambient Temperature at 2m [C]"
humidity = "Relative Humidity at 2m [%]"
wind_speed = "Wind Speed at 10m [m/s]"
weather_data.plot(y=[humidity,temperature])

In [ ]:
weather_data.plot.scatter(x=temperature,y=humidity,color=wind_speed)

Podemos estudiar la correlación entre las variables con la **matriz de correlación**. En este caso, se ve que la humedad y la temperatura tienen una correlación negativa débil y la probabilidad de precipitación con la humedad, positiva y débil. Los valores de esta matriz van desde -1 a 1: -1 para correlación total negativa, y 1 para correlación total positiva (por eso en la diagonal hay sólo valores 1). Esto lo podemos comparar con sus gráficas.

In [6]:
weather_data.corr(numeric_only=True)

,Air Pressure at 2m [hPa],Ambient Temperature at 2m [C],Dew Temperature at 2m [C],Relative Humidity at 2m [%],Wind Direction at 10m [deg],Wind Speed at 10m [m/s]
Air Pressure at 2m [hPa],1.000000,0.642781,0.572242,-0.421586,0.198645,-0.445494
Ambient Temperature at 2m [C],0.642781,1.000000,0.823481,-0.735613,0.330278,-0.570567
Dew Temperature at 2m [C],0.572242,0.823481,1.000000,-0.221443,0.349599,-0.547747
Relative Humidity at 2m [%],-0.421586,-0.735613,-0.221443,1.000000,-0.150277,0.326709
Wind Direction at 10m [deg],0.198645,0.330278,0.349599,-0.150277,1.000000,-0.363432
Wind Speed at 10m [m/s],-0.445494,-0.570567,-0.547747,0.326709,-0.363432,1.000000


A esta tabla le podemos agregar columnas, por ejemplo, la temperatura del punto de rocío $T_{dp}$, con $T$ la temperatura y $RH$ la humedad relativa. Esta se calcula con $T_{dp} = T-(100-RH)/5$.

In [7]:
weather_data['dew_point']=weather_data[temperature]-(100-weather_data[humidity])/5

In [8]:
weather_data.describe()

,Air Pressure at 2m [hPa],Ambient Temperature at 2m [C],Dew Temperature at 2m [C],Relative Humidity at 2m [%],Wind Direction at 10m [deg],Wind Speed at 10m [m/s],dew_point
count,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,769.32744,10.734745,-5.635275,18.149900,68.884450,8.560755,-5.635275
std,1.64387,4.190231,2.910737,12.189121,90.373276,3.818203,2.910737
min,766.50000,1.300000,-11.300000,1.000000,0.000000,0.100000,-11.300000
25%,768.00000,7.400000,-8.200000,10.000000,24.000000,5.900000,-8.200000
50%,768.90000,9.800000,-5.900000,15.000000,41.000000,8.600000,-5.900000
75%,770.80000,14.700000,-3.100000,24.000000,54.000000,11.100000,-3.100000
max,773.00000,19.500000,1.500000,83.000000,359.000000,24.200000,1.500000


In [ ]:
weather_data.plot()

In [ ]:
weather_data.plot.scatter(x=temperature,y=humidity,color='dew_point')

In [11]:
weather_data.corr()

,Air Pressure at 2m [hPa],Ambient Temperature at 2m [C],Dew Temperature at 2m [C],Relative Humidity at 2m [%],Wind Direction at 10m [deg],Wind Speed at 10m [m/s],dew_point
Air Pressure at 2m [hPa],1.000000,0.642781,0.572242,-0.421586,0.198645,-0.445494,0.572242
Ambient Temperature at 2m [C],0.642781,1.000000,0.823481,-0.735613,0.330278,-0.570567,0.823481
Dew Temperature at 2m [C],0.572242,0.823481,1.000000,-0.221443,0.349599,-0.547747,1.000000
Relative Humidity at 2m [%],-0.421586,-0.735613,-0.221443,1.000000,-0.150277,0.326709,-0.221443
Wind Direction at 10m [deg],0.198645,0.330278,0.349599,-0.150277,1.000000,-0.363432,0.349599
Wind Speed at 10m [m/s],-0.445494,-0.570567,-0.547747,0.326709,-0.363432,1.000000,-0.547747
dew_point,0.572242,0.823481,1.000000,-0.221443,0.349599,-0.547747,1.000000


La correlación entre el punto de rocío y la humedad es alta.